### Datos en Bases de datos SQL

La mayoría de las empresas guardan gran cantidad de datps en BBDD relaciones. Aquí van unas nocios para poder "manejar" estos datos cuando tengamos esas entradas en proyectos de Big Data. En clusters de Cloudera, es común encontrarse tablas SQL en HIVE o IMPALA.

In [1]:
import sqlite3
import pandas as pd 

In [2]:
# Conectar a la base de datos (se crea si no existe)
conn = sqlite3.connect('test.db') # Esto nos genera un nuevo fichero (en caso de que no exista) de base de datos formato sqlite
cursor = conn.cursor() # Necesario para realizar conexión base de datos con notebook y poder realizar queries

In [3]:
# Crear tabla
cursor.execute("""
               CREATE TABLE IF NOT EXISTS usuarios (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               nombre TEXT NOT NULL,
               edad INTEGER,
               email TEXT UNIQUE
            )
""")
conn.commit() # Debe venir siempre acompañado de esta función para que queden actualizadas en la base de datos

In [4]:
# Insertar datos de ejemplo
usuarios = [
    ('Alice',25,'alice@example.com'),
    ('Bob',30,'bob@example.com'),
    ('Charlie',22,'charlie@example.com')
]

In [5]:
cursor.executemany('INSERT INTO usuarios (nombre,edad,email) VALUES (?,?,?)',usuarios) # Introducimos los datos creados como variable usuarios sobre la tabla usuarios de text.db (directorio locla sqlite)
conn.commit()

In [6]:
# Leer los datos
cursor.execute('SELECT * FROM usuarios')
datos = cursor.fetchall()

In [7]:
datos

[(1, 'Alice', 25, 'alice@example.com'),
 (2, 'Bob', 30, 'bob@example.com'),
 (3, 'Charlie', 22, 'charlie@example.com')]

In [8]:
# Convertir a dataframe

df = pd.DataFrame(datos, columns = ['id','nombre','edad','email'])
df

,id,nombre,edad,email
0,1,Alice,25,alice@example.com
1,2,Bob,30,bob@example.com
2,3,Charlie,22,charlie@example.com


In [9]:
query = 'SELECT * FROM usuarios' # Definimos la query

In [10]:
df_pandas = pd.read_sql_query(query,conn,index_col='id')
df_pandas.head()

,nombre,edad,email
id,,,
1,Alice,25,alice@example.com
2,Bob,30,bob@example.com
3,Charlie,22,charlie@example.com


In [11]:
query = 'SELECT * FROM usuarios WHERE edad > 25' # También le podemos aplicar filtros a la query

In [12]:
df_pandas = pd.read_sql_query(query,conn,index_col='id')
df_pandas.head()

,nombre,edad,email
id,,,
2,Bob,30,bob@example.com


In [13]:
# Cerrar la conexión
conn.close() 

El objetivo es pasar la base de datos a un dataframe de pandas

### Datos en XML

In [ ]:
# Método 1
with open (r'C:\Users\pauri\OneDrive\Escritorio\4) MÁSTER EN DATA SCIENCE, BIG DATA & BUSINESS ANALYTICS (UCM)\MÓDULO 8_MACHINE LEARNING\Clase 1 y 2\2. INGESTA DE DATOS\prueba.xml','r') as f:
    print(f.read())

<data>
    <student name="John">
        <email>john@mail.com</email>
        <grade>A</grade>
        <age>16</age>
    </student>
    <student name="Alice">
        <email>alice@mail.com</email>
        <grade>B</grade>
        <age>17</age>
    </student>
    <student name="Bob">
        <email>bob@mail.com</email>
        <grade>C</grade>
        <age>16</age>
    </student>
    <student name="Hannah">
        <email>hannah@mail.com</email>
        <grade>A</grade>
        <age>17</age>
    </student>
</data>


In [ ]:
# Método 2
df1 = pd.read_xml(r'C:\Users\pauri\OneDrive\Escritorio\4) MÁSTER EN DATA SCIENCE, BIG DATA & BUSINESS ANALYTICS (UCM)\MÓDULO 8_MACHINE LEARNING\Clase 1 y 2\2. INGESTA DE DATOS\prueba.xml')

In [15]:
df1.head()

,name,email,grade,age
0,John,john@mail.com,A,16
1,Alice,alice@mail.com,B,17
2,Bob,bob@mail.com,C,16
3,Hannah,hannah@mail.com,A,17


In [22]:
# Método 3
import xml.etree.ElementTree as et

xtree = et.parse(r'C:\Users\pauri\OneDrive\Escritorio\4) MÁSTER EN DATA SCIENCE, BIG DATA & BUSINESS ANALYTICS (UCM)\MÓDULO 8_MACHINE LEARNING\Clase 1 y 2\2. INGESTA DE DATOS\prueba.xml')
xroot = xtree.getroot()
xroot

<Element 'data' at 0x000001BE6687DA80>

In [23]:
# Lo guardamos en un DataFrame

df_cols =  ['name','email','grade','age']
rows = []

for node in xroot:
    s_name = node.attrib.get('name')
    s_email = node.find('email').text
    s_grade = node.find('grade').text
    s_age = node.find('age').text
    rows.append({'name':s_name,'email':s_email,
                 'grade':s_grade,'age':s_age,})
    
out_df = pd.DataFrame(rows,columns = df_cols)
out_df

,name,email,grade,age
0,John,john@mail.com,A,16
1,Alice,alice@mail.com,B,17
2,Bob,bob@mail.com,C,16
3,Hannah,hannah@mail.com,A,17


### WebScraping html WIKIPEDIA

Nos permite extraer información de paginas web (Método 1. extraer de la propia página, Método 2. de APIs)

In [24]:
import requests # Para hacer peticiones HTTP
from bs4 import BeautifulSoup # Para parsear y extraer información del HTML descargado

In [34]:
resp = requests.get('https://web.gw.fotocasa.es/v2/propertysearch/searchurl?combinedLocationIds=724,14,28,173,0,28079,0,0,0&culture=es-ES&latitude=40.4096&longitude=-3.68624&text/',
                    headers = {'User-Agent':'chrome'})

In [ ]:
resp.json()

{'location': 'madrid-capital', 'zone': 'todas-las-zonas', 'text': ''}

In [45]:
url = 'https://es.wikipedia.org/wiki/Espa%C3%B1a'

In [46]:
peticion = requests.get(url)

In [47]:
scraping = BeautifulSoup(peticion.text,'lxml')

In [48]:
scraping

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>España - Wikipedia, la enciclopedia libre</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-ena

In [51]:
# Queremos extraer todos los encabezados

for sub_heading in scraping.find_all(['h2','h1']):
    print(sub_heading.text)

Contenidos
España
Toponimia
Historia
Política y administración
Geografía
Demografía
Economía
Infraestructura y servicios
Cultura
Véase también
Notas
Referencias
Bibliografía
Enlaces externos


In [54]:
url2 = 'https://www.nationmaster.com/country-info/stats/Media/Internet-users'
res = requests.get(url2)
soup = BeautifulSoup(res.content,'lxml')

In [64]:
table = soup.find_all('table')[0]
table.find_all('tr')

[<tr>
 <th class="thno" scope="col">#</th>
 <th scope="col"><span class="countrycol">COUNTRY</span></th>
 <th class="sorter-formatted-float amount" scope="col">AMOUNT</th>
 <th class="date" scope="col">DATE</th>
 <th class="sorter-formatted-float" scope="col">GRAPH</th>
 <th scope="col">HISTORY</th>
 </tr>,
 <tr>
 <td class="tblno" data-row="1">1</td>
 <td>
 <a data-event-action="Click country" href="/country-info/profiles/China/Media"><i class="flag flag-cn"></i><span class="full">China</span></a>
 </td>
 <td class="amount" data-raw="389000000.0">
                                 389 million
                             </td>
 <td class="date">2009</td>
 <td data-raw="389000000.0">
 <span class="graph">
 <span class="bar" style="width: 100.0000%"></span>
 </span>
 </td>
 <td>
 <span class="spark" values="2009:389000000.0"></span>
 </td>
 </tr>,
 <tr>
 <td class="tblno" data-row="2">2</td>
 <td>
 <a data-event-action="Click country" href="/country-info/profiles/United-States/Media"><i 

In [69]:
df = pd.read_html(str(table))
df[0]

C:\Users\pauri\AppData\Local\Temp\ipykernel_12640\111615480.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))


,#,COUNTRY,AMOUNT,DATE,GRAPH,HISTORY
0,1,China,389 million,2009,NaN,NaN
1,2,United States,245 million,2009,NaN,NaN
2,3,Japan,99.18 million,2009,NaN,NaN
3,NaN,Group of 7 countries (G7) average (profile),80.32 million,2009,NaN,NaN
4,4,Brazil,75.98 million,2009,NaN,NaN
...,...,...,...,...,...,...
244,214,Niue,1100,2009,NaN,NaN
245,=215,"Saint Helena, Ascension, and Tristan da Cunha",900,2009,NaN,NaN
246,=215,Saint Helena,900,2009,NaN,NaN
247,217,Tokelau,800,2008,NaN,NaN
